In [1]:
#!/usr/bin/env python
import numpy as np
import os
import sys
import glob
import time
from scipy import misc
import caffe

def prewhiten(x):
    mean = np.mean(x)
    std = np.std(x)
    std_adj = np.maximum(std, 1.0/np.sqrt(x.size))
    y = np.multiply(np.subtract(x, mean), 1/std_adj)
    return y

def max_diff(layer_1, layer_2):
    diff_array = np.absolute(np.subtract(layer_1, layer_2))
    max_d = np.amax(diff_array)
    print ('Max abs diff: ', max_d)
    return max_d

def read_images(input_image_names, io = 'caffe', output= 'caffe', transpose = (2, 0, 1)):
    if io=='caffe':
        inputs =[prewhiten(caffe.io.load_image(im_f)) for im_f in input_image_names]
        if output == 'caffe':
            transformed_images = [image.transpose(transpose) for image in inputs]#transformer.preprocess('data', image)
        else:
            transformed_images = inputs
        inputs = transformed_images
    elif io == 'misc':
        inputs =[prewhiten(misc.imread(im_f)) for im_f in input_image_names] 
        if output == 'caffe':
            transformed_images = [image.transpose(transpose) for image in inputs]#transformer.preprocess('data', image)
        else:
            transformed_images = inputs
        inputs = transformed_images
    else:
        print ('Invaid IO mode...!')
    return inputs

def get_embeddings_caffe(inputs, output_layer = 'normed_features', model_def = '/home/ovuser/Projects/tf_2_caffe/facenet_deploy.prototxt', pretrained_model = '/home/ovuser/Projects/tf_2_caffe/inc_resnet_v1_facenet.caffemodel', gpu = True):
    #pycaffe_dir = os.path.dirname(__file__)
    if gpu:
        caffe.set_mode_gpu()
        print("GPU mode")
    else:
        caffe.set_mode_cpu()
        print("CPU mode")

    # Make classifier.
    classifier = caffe.Net(model_def, pretrained_model, caffe.TEST)
    print("Classifying %d inputs." % len(inputs))    
    outputs = []
    start = time.time()
    for transformed_image in inputs:
        # Classify.
        # copy the image data into the memory allocated for the net
        classifier.blobs['data'].data[...] = transformed_image
        ### perform classification
        output = classifier.forward()[output_layer][0].copy()
        outputs.append(output)
    print("Done in %.2f s." % (time.time() - start))
    return outputs

def tf_fwd_pass(images, output_layers, model_dir = '/home/ovuser/FaceRecognition/models/20170512-110547'):
    import tensorflow as tf
    import facenet
    outputs = []
    with tf.Graph().as_default():
        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
        sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
        with sess.as_default():
            #Load the model
            facenet.load_model(model_dir)
            # Get input and output tensors
            images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
            phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
            feed_dict = { images_placeholder: images, phase_train_placeholder:False }
            for tensor_name in output_layers:
                tensor = tf.get_default_graph().get_tensor_by_name(tensor_name) 
                outputs.append(sess.run(tensor, feed_dict=feed_dict))
    return outputs

In [2]:
import os
dataset = '/home/ovuser/Pictures/panel-data-aligned/'
subjects = [dataset+subject for subject in os.listdir(dataset)]
input_image_names = []
for subject in subjects:
    input_image_names.extend([subject+'/'+image for image in os.listdir(subject)])

In [8]:
caffe_images = read_images(input_image_names, 'misc', 'caffe')
tf_images = read_images(input_image_names, 'misc', 'tf')

In [ ]:
caffe_images = read_images(['/home/ovuser/Pictures/panel-data-aligned/deubanks/deubanks_41.png'], 'caffe', 'caffe')
tf_images = read_images(['/home/ovuser/Pictures/panel-data-aligned/deubanks/deubanks_41.png'], 'caffe', 'tf')

In [9]:
caffe_outputs = get_embeddings_caffe(caffe_images[:10], output_layer = 'normed_features', model_def = '/home/ovuser/Projects/tf_2_caffe/facenet_deploy.prototxt', pretrained_model = '/home/ovuser/Projects/tf_2_caffe/inc_resnet_v1_facenet_latest_5_15.caffemodel')

GPU mode
Classifying 10 inputs.
Done in 0.55 s.


In [10]:
tf_layers = ['embeddings:0']

In [11]:
tf_outputs = tf_fwd_pass(tf_images[:10], tf_layers)

Model directory: /home/ovuser/FaceRecognition/models/20170512-110547
Metagraph file: model-20170512-110547.meta
Checkpoint file: model-20170512-110547.ckpt-250000
'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from /home/ovuser/FaceRecognition/models/20170512-110547/model-20170512-110547.ckpt-250000


In [12]:
diff = []
for i in range(len(caffe_outputs)):
    print (input_image_names[i])
    print (np.array_equal(tf_images[i], caffe_images[i].transpose(1,2,0)))
    #print (caffe_outputs[i].shape, tf_outputs[0][i].shape)
    #print (caffe_outputs[i][:10], tf_outputs[0][i][:10])
    diff.append(max_diff(caffe_outputs[i], tf_outputs[0][i]))

/home/ovuser/Pictures/panel-data-aligned/deubanks/deubanks_41.png
True
('Max abs diff: ', 1.296401e-05)
/home/ovuser/Pictures/panel-data-aligned/deubanks/deubanks_22.png
True
('Max abs diff: ', 1.5607104e-05)
/home/ovuser/Pictures/panel-data-aligned/deubanks/deubanks_7.png
True
('Max abs diff: ', 1.50753185e-05)
/home/ovuser/Pictures/panel-data-aligned/deubanks/deubanks_34.png
True
('Max abs diff: ', 2.6457012e-05)
/home/ovuser/Pictures/panel-data-aligned/deubanks/deubanks_19.png
True
('Max abs diff: ', 1.1220574e-05)
/home/ovuser/Pictures/panel-data-aligned/deubanks/deubanks_30.png
True
('Max abs diff: ', 1.316797e-05)
/home/ovuser/Pictures/panel-data-aligned/deubanks/deubanks_18.png
True
('Max abs diff: ', 1.0591e-05)
/home/ovuser/Pictures/panel-data-aligned/deubanks/deubanks_26.png
True
('Max abs diff: ', 1.6897917e-05)
/home/ovuser/Pictures/panel-data-aligned/deubanks/deubanks_31.png
True
('Max abs diff: ', 1.4953315e-05)
/home/ovuser/Pictures/panel-data-aligned/deubanks/deubanks_1

In [ ]:
print (np.sum(np.square(caffe_outputs[0])), np.sum(np.square(tf_outputs[0][0])))

In [ ]:
print (input_image_names[:10])

In [ ]:
diff = []
for i in range(len(caffe_outputs)):
    print (input_image_names[i])
    print (np.array_equal(tf_images[i], caffe_images[i].transpose(1,2,0)))
    #print (caffe_outputs[i].shape, tf_outputs[0][i].shape)
    #print (caffe_outputs[i][:10], tf_outputs[0][i][:10])
    diff.append(max_diff(caffe_outputs[i], tf_outputs[0][i]))

In [ ]:
deubanks_41_output_caffe = caffe_outputs[0]
deubanks_41_output_tf = tf_outputs[0][0]

In [ ]:
print max_diff(deubanks_41_output_caffe, deubanks_41_output_tf)

In [ ]:
print max_diff(deubanks_41_output_caffe, caffe_outputs[0])

In [ ]:
print max_diff(deubanks_41_output_tf, tf_outputs[0])

In [ ]:
def verify_individual_fwd_passes(input_image_names):
    for image in input_image_names:
        caffe_images = read_images([image], 'caffe', 'caffe')
        tf_images = read_images([image], 'caffe', 'tf')
        caffe_outputs = get_embeddings_caffe(caffe_images, output_layer = 'normed_features', model_def = '/home/ovuser/Projects/tf_2_caffe/facenet_deploy.prototxt', pretrained_model = '/home/ovuser/Projects/tf_2_caffe/inc_resnet_v1_facenet_latest_5_15.caffemodel')
        tf_outputs = tf_fwd_pass(tf_images, ['embeddings:0'])
        print (max_diff(caffe_outputs[0], tf_outputs[0][0]))

In [ ]:
verify_individual_fwd_passes(input_image_names[:20])